Assignement 3 trial

In [ ]:
import torch
import copy
import numpy as np
import itertools
import random as rd
from matplotlib import pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [ ]:
import matplotlib

matplotlib.use("TkAgg")
BOLD = "\033[1m"
RESET = "\033[0m"

In [ ]:
#defining environment

class Environment():
    
    #fields:
    #a grid of size defined by parameter
    #b, a tuple repsenting the delivery location
    #a, a tuple representing the package location
    #size, an int representing the horizontal and vertical dimensions of the grid
    def __init__(self, size:int):
        self.size = size
        self.actions = ['north', 'south', 'west', 'east']
        self.reward_pickup = 50
        self.reward_handover = 200
        self.a1_actual_reward=0
        self.a2_actual_reward=0
        self.reward_deliver=300 
        self.reward_move = -1.5
        self.reset_env()

       
    def reset_env(self):
        combinations = list(itertools.product(list(range(self.size)), repeat=2))
        rd.shuffle(combinations)
        self.a, self.agent2,self.b, self.agent_coords = combinations[0:4]
        self.collected_agent1 = 0
        self.collected_agent2 = 0
        self.done = False
        self.terminated = False
        self.handover=0
        self.test_length = 0 #an additional parameter to determine how long the environment has been active for
        self.setup_grid()

    # def coordinates_to_noisy_ohv(self, coordinate:int):
    #     ohv = np.zeros(4)
    #     noise = np.random.normal(-1e-4, 1e-4, ohv.shape)
    #     ohv[coordinate] = 1
    #     ohv = ohv + noise
    #     return ohv
        
    # def get_travel_distance(self,handover):
    #     agent1_travel=abs(self.agent_coords[0]-self.a[0]) + abs(self.agent_coords[1]-self.a[1])
    #     agent2_travel=abs(self.agent2[0]-handover[0]) + abs(self.agent2[1]-handover[1])

    #     return agent1_travel,agent2_travel

    
    def get_optimal_distance(self):
        return abs(self.agent_coords[0]-self.a[0]) + abs(self.agent_coords[0]-self.a[1]) + abs(self.a[0]-self.b[0]) + abs(self.a[1]-self.b[1])


    def setup_grid(self):
        grid = np.zeros((self.size, self.size))
        self.grid = grid
        grid[self.a] = 1#package
        grid[self.b] = 2#detsination
        grid[self.agent_coords] = -1
        grid[self.agent2]= 3 #agent2

    def get_state(self):
        return np.array([self.agent_coords[0], self.agent_coords[1], self.collected, self.a[0], self.a[1],self.agent2[0],self.agent2[1], self.b[0], self.b[1]])
    
    #Method which updates the location of the agent on the grid. Currently just zeroes whatever it landed on - can include other logic instead
    def move_agent(self, action):
        #For agent move, note that 0 = up, 1=down, 2=left, 3=right
        self.test_length += 1
        distance_agent1 = 0
        distance_agent2 = 0

        #Assign new coordinates for agent1 to exist at
        new_x, new_y = self.agent_coords
        if action == 'north' and self.agent_coords[1] > 0:
            new_y -= 1
        elif action == 'south' and self.agent_coords[1] < self.size - 1:
            new_y += 1
        elif action == 'west' and self.agent_coords[0] > 0:
            new_x -= 1
        elif action == 'east' and self.agent_coords[0] < self.size - 1:
             new_x += 1
         #Assign new coordinates for agent2 to exist at
        new_a2x, new_a2y = self.agent2
        if action == 'north' and self.agent2[1] > 0:
            new_a2y -= 1
        elif action == 'south' and self.agent2[1] < self.size - 1:
            new_a2y += 1
        elif action == 'west' and self.agent2[0] > 0:
            new_a2x -= 1
        elif action == 'east' and self.agent2[0] < self.size - 1:
             new_a2x += 1
        
        
        #Update the grid based on new agent coordinates
        self.grid[self.agent_coords] = 0
        self.agent_coords = (new_x, new_y)
        distance_agent1 += 1
        self.grid[self.agent_coords] = -1

          #Update the grid based on new agent coordinates
        self.grid[self.agent2] = 0
        self.agent2 = (new_a2x, new_a2y)
        distance_agent2 += 1
        self.grid[self.agent2] = 3

        #Handle logic based on new agent location
        if self.collected_agent1 == 0: # pick up for agent 1
            if self.agent_coords == self.a:
                self.collected_agent1 = 1
                reward_agent1 = self.reward_pickup
            else:
                reward_agent1 = distance_agent1**2
        elif self.collected_agent2 == 0 and self.collected_agent1 == 1: # agent handover
            if self.agent_coords == self.agent2:
                self.collected_agent2 = 1
                self.collected_agent1 = 0
                reward_agent1 = self.reward_handover - (distance_agent1 ** 2) # reward for agent 1 for handover
                reward_agent2 = self.reward_handover - (distance_agent2 ** 2) # reward for agent 2 for handover
                distance_agent2 = 0 # check if needed
            else:
                reward_agent1 = distance_agent1**2
                reward_agent2 = distance_agent2**2
        else: # agent 2 is making delivery
            if self.agent2 == self.b:
                self.done = True
                reward_agent2 = self.reward_deliver
            else:
                reward_agent2 = distance_agent2**2

                #This code was considered test code which could be useful in exiting early
        if self.test_length > 70:
            self.terminated = True
        return reward_agent1, reward_agent2, self.done, self.terminated

    def get_grid(self):
        return self.grid.tolist()
    
    def setup_grid(self):
        self.grid = np.zeros((self.size, self.size))
        # Place the "home" location (0) on the grid
        x, y = self.agent_coords
        self.grid[0,0]=1 #pickup
        self.grid[self.size-1,self.size-1]=2 #destination
        self.grid[x, y] = 0
    
    def plot_grid(self, snapshot, ax=None):
        if ax is None:
            fig, ax = plt.subplots()
            ax.set_facecolor('white')
        else:
            ax.clear()
        
        # Plot the grid
        ax.imshow(np.array([[0]]), cmap="bone", extent=[0, self.size, 0, self.size])

        for i in range(self.size):
            for j in range(self.size):
                cell_value = snapshot[i][j]
                if cell_value == -1:
                    # Display agent image in the cell
                    imagebox = OffsetImage(agent1_img, zoom=0.08)
                    ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)
                elif cell_value == 1:
                    # Display package image in the cell
                    imagebox = OffsetImage(package_img, zoom=0.03)
                    ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)
                elif cell_value == 2:
                    # Display destination image in the cell
                    imagebox = OffsetImage(destinationB_img, zoom=0.05)
                    ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)
                elif cell_value == 3:
                    # Display agent2 image in the cell
                    imagebox = OffsetImage(agent2_img, zoom=0.05)
                    ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)
                else:
                    ax.text(j + 0.5, self.size - i - 0.5, self.grid[i, j], ha='center', va='center', fontsize=20, color='black')
        
        # Set axis properties
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_xticks(np.arange(self.size) + 1)
        ax.set_yticks(np.arange(self.size) + 1)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.grid(True, linewidth=2, color='white')
        
        # Set title
        ax.set_title("Package Delivery Agent")

        return ax

agent1_img = plt.imread('agent.jpg')
agent2_img=plt.imread('agent2.png')
package_img = plt.imread('package.jpg')
destinationB_img = plt.imread('destinationB.jpg')




In [ ]:
class DQN:
    def __init__(self, state_size,action_size = 4):
        l1 = state_size
        l2 = 24
        l3 = 24
        l4 = action_size
        self.model = torch.nn.Sequential(
        torch.nn.Linear(l1, l2),
        torch.nn.ReLU(),
        torch.nn.Linear(l2, l3),
        torch.nn.ReLU(),
        torch.nn.Linear(l3,l4))

        self.model2 = copy.deepcopy(self.model)
        self.model2.load_state_dict(self.model.state_dict())
        self.loss_fn = torch.nn.MSELoss()
        self.learning_rate = 0.001
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

# The function "update_target" copies the state of the prediction network to the target network. You need to use this in regular intervals.
    def update_target(self):
        self.model2.load_state_dict(self.model.state_dict())

# The function "get_qvals" returns a numpy list of qvals for the state given by the argument based on the prediction network.
    def get_qvals(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        q_values = self.model(state)
        return q_values

# The function "get_maxQ" returns the maximum q-value for the state given by the argument based on the target network.
    def get_maxQ(self,state):
        q_values = self.model(state)
        return torch.max(q_values).item()
    
    

# The function "train_one_step_new" performs a single training step.
# It returns the current loss (only needed for debugging purposes).
# Its parameters are three parallel lists: a minibatch of states, a minibatch of actions,
# a minibatch of the corresponding TD targets and the discount factor.
    def train_one_step(self, states, actions, targets):
        targets_reply = []
        state1_batch = torch.cat([torch.from_numpy(s).float() for s in states])
        action_batch = torch.Tensor(actions)
        Q1 = self.model(state1_batch)
        X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
        Y = torch.tensor(targets).float()
        loss = self.loss_fn(X, Y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [ ]:
class TransportAgent1:
    def __init__(self, statespace_size=7, gamma=0.99, learning_rate=0.997, start_epsilon=1.0,
                 epsilon_decay_factor=0.999, min_epsilon=0.05, replay_buffer_size=1000,
                 batch_size=200, network_copy_frequency=500):
        self.statespace_size = statespace_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.model2 = DQN()
        self.memory = []  # Using a list instead of a deque
        self.epsilon = start_epsilon
        self.epsilon_min = min_epsilon
        self.epsilon_decay_factor = epsilon_decay_factor
        self.replay_buffer_size = replay_buffer_size
        self.batch_size = batch_size
        self.network_copy_frequency = network_copy_frequency
        self.steps_since_copy = 0  # Counter for network copy

    def remember(self, state, action, reward, next_state, done):
            self.memory.append((state, action, reward, next_state, done))
            while len(self.memory) > self.replay_buffer_size:
                self.memory.pop(0)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(range(4))  # Random action
        q_values = self.get_qvals(state)
        return np.argmax(q_values)  # Greedy action

    def process_minibatch(self, minibatch):
        states = []
        actions = []
        targets = []
        for state, action, reward, next_state, done in minibatch:
            q_values = DQN.get_qvals(state)
            if done:
                q_values[action] = reward
            else:
                q_values[action] = reward + self.gamma * DQN.get_maxQ(next_state)
            states.append(state)
            actions.append(action)
            targets.append(q_values[action])
        return np.array(states), np.array(actions), np.array(targets) #this is returning a thruple of state transitions?
    
    def save_model_parameters(self, destination):
        torch.save(DQN.model2.state_dict(), destination)

    def load_model_parameters(self, path):
        DQN.model2.load_state_dict(torch.load(path))


    

In [ ]:
class TransportAgent2:
    def __init__(self, statespace_size=7, gamma=0.99, learning_rate=0.997, start_epsilon=1.0,
                 epsilon_decay_factor=0.999, min_epsilon=0.05, replay_buffer_size=1000,
                 batch_size=200, network_copy_frequency=500):
        self.statespace_size = statespace_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.model3 = DQN()
        self.memory = []  # Using a list instead of a deque
        self.epsilon = start_epsilon
        self.epsilon_min = min_epsilon
        self.epsilon_decay_factor = epsilon_decay_factor
        self.replay_buffer_size = replay_buffer_size
        self.batch_size = batch_size
        self.network_copy_frequency = network_copy_frequency
        self.steps_since_copy = 0  # Counter for network copy

    def remember(self, state, action, reward, next_state, done):
            self.memory.append((state, action, reward, next_state, done))
            while len(self.memory) > self.replay_buffer_size:
                self.memory.pop(0)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(range(4))  # Random action
        q_values = self.get_qvals(state)
        return np.argmax(q_values)  # Greedy action

    def process_minibatch(self, minibatch):
        states = []
        actions = []
        targets = []
        for state, action, reward, next_state, done in minibatch:
            q_values = DQN.get_qvals(state)
            if done:
                q_values[action] = reward
            else:
                q_values[action] = reward + self.gamma * DQN.get_maxQ(next_state)
            states.append(state)
            actions.append(action)
            targets.append(q_values[action])
        return np.array(states), np.array(actions), np.array(targets) #this is returning a thruple of state transitions?
    
    def save_model_parameters(self, destination):
        torch.save(DQN.model2.state_dict(), destination)

    def load_model_parameters(self, path):
        DQN.model2.load_state_dict(torch.load(path))


    